In [28]:
%%writefile Classify1_tumor.py

# -----------------------------
# 📚 Import Libraries
# -----------------------------
import streamlit as st
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# -----------------------------
# 🧠 Load EfficientNet Model
# -----------------------------
model = load_model("resnet_tumor_model.keras")

# -----------------------------
# 🖼️ Helper Functions
# -----------------------------
def prepare_image(image, target_size=(224, 224)):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

def predict_image(model, image):
    img = prepare_image(image)
    pred = model.predict(img)
    class_idx = np.argmax(pred)
    classes = {0: "No Tumor ❌", 1: "Tumor ✅"}
    return classes[class_idx], pred

# -----------------------------
# 🌟 Streamlit Page Config
# -----------------------------
st.set_page_config(
    page_title="Brain Tumor Detection 🧠",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Sidebar
st.sidebar.title("⚙️ Options")
st.sidebar.write("This app detects brain tumors in MRI images.")
st.sidebar.markdown("Upload a clear MRI image for best results.")

# Main Page
st.title("🧠 Brain Tumor Detection")
st.markdown("Upload a brain MRI image and the model will predict if a tumor is present.")

# -----------------------------
# 📤 File Uploader
# -----------------------------
uploaded_file = st.file_uploader("Choose an image...", type=["jpg","png","jpeg"])
if uploaded_file is not None:
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

    st.image(image, caption='Uploaded Image', width=100)
    st.write("---")

    # -----------------------------
    # 🎯 Prediction
    # -----------------------------
    result, prob = predict_image(model, image)

    st.markdown(f"### Prediction: **{result}**")
    
    # Probability bars
    classes = ["No Tumor ❌", "Tumor ✅"]
    probs = prob[0]

    st.markdown("### Probability Scores")
    for c, p in zip(classes, probs):
        st.progress(int(p*100))
        st.write(f"{c}: {p*100:.2f}%")

    # Optional: Probability bar chart
    plt.figure(figsize=(5,5))
    fig, ax = plt.subplots()
    ax.bar(classes, probs, color=["green", "red"])
    ax.set_ylim([0, 1])
    ax.set_ylabel("Probability")
    ax.set_title("Prediction Probabilities")
    st.pyplot(fig)


Writing Classify1_tumor.py


^C


In [ ]:
! streamlit run Classify1_tumor.py


In [ ]:
segmentation